In [1]:
#imports for news site scraping and writing out json
import newspaper
import json

In [2]:
#grab articles from CNN and check how many we got
#the memoize_articles=False is there to not allow newspaper library to cache results as it was
#not working well across multiple runs
cnn_paper = newspaper.build('https://www.cnn.com', memoize_articles=False)
len(cnn_paper.articles)

845

In [3]:
#create wrapping dictionary, for ease when writing out to json
output = {}
for article in cnn_paper.articles:
    #tmp will be our actual json object
    tmp = {}
    #attempt to download the article
    try:
        article.download()
        article.parse()
    #if download fails just go to the next one
    except:
        continue
    #set attributes of the json object based on contents of article
    tmp['title'] = article.title
    tmp['authors'] = article.authors
    try:
        #the line below is just some formatting for json as it cannot take a python datetime object
        tmp['publish_date'] = "{}-{}-{}".format(article.publish_date.year,article.publish_date.month,article.publish_date.day)
    except AttributeError:
        #if no publish date just set the object date to none
        print("Date Not Available")
        tmp['publish_date'] = None
    tmp['text'] = article.text
    tmp['url'] = article.url
    #store the json object in our wrapper dict under the articles title (couldn't think of a better way to identify articles)
    output[tmp['title']] = (tmp)
    #stop after 50 articles
    if len(output) is 50:
        break

Date Not Available
Date Not Available


In [4]:
#verify we have 50 articles
len(output)

50

In [5]:
#open our file as writeable
with open("output.json", "w") as outfile:
    #some formatting for json
    outfile.write("{\n")
    i = 0
    #for each title in our wrapper dict
    for title in output:
        #write the title and some necessary formatting
        outfile.write("\"{}\":".format(title))
        #write the actual dict containing the article information
        outfile.write(json.dumps(output[title]))
        #if-else is just to check whether comma is needed
        if i is not 49:
            outfile.write(",\n")
        else:
            outfile.write("\n")
        i += 1
    outfile.write("}")